In [1]:
import pandas as pd
import folium
from pathlib import Path
from datetime import datetime

In [2]:
gtfs_path = Path("..") / "data" / "raw" / "budapest_gtfs"

In [3]:
# Load GTFS core files
stops = pd.read_csv(gtfs_path / "stops.txt")
routes = pd.read_csv(gtfs_path / "routes.txt")
trips = pd.read_csv(gtfs_path / "trips.txt")
stop_times = pd.read_csv(gtfs_path / "stop_times.txt")

In [4]:
# Filter for line 4 or 6 tram
target_routes = routes[routes["route_short_name"].isin(["2"])]
target_route_ids = target_routes["route_id"].tolist()

In [5]:
# Get a single trip from today's service
target_trips = trips[trips["route_id"].isin(target_route_ids)]
sample_trip_id = target_trips.iloc[0]["trip_id"]

In [6]:
# Get stop_times for that trip
trip_stop_times = stop_times[stop_times["trip_id"] == sample_trip_id].sort_values("stop_sequence")

In [7]:
# Merge with stop coordinates
trip_stops = pd.merge(trip_stop_times, stops, on="stop_id")
trip_stops["arrival_time"] = pd.to_timedelta(trip_stops["arrival_time"])

In [8]:
# Assume start of simulation is current time
sim_start = datetime.now().replace(second=0, microsecond=0)
first_stop_offset = trip_stops["arrival_time"].iloc[0]
trip_stops["sim_time"] = [sim_start + (t - first_stop_offset) for t in trip_stops["arrival_time"]]

In [9]:
# Create base map
start_coords = [trip_stops.iloc[0]["stop_lat"], trip_stops.iloc[0]["stop_lon"]]
sim_map = folium.Map(location=start_coords, zoom_start=13)

In [10]:
# Plot all stops
for _, row in trip_stops.iterrows():
    folium.Marker(
        location=[row["stop_lat"], row["stop_lon"]],
        popup=f"{row['stop_name']}<br>ETA: {row['sim_time'].strftime('%H:%M')}",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(sim_map)

In [11]:
# Simulate vehicle icon moving along the stops (showing current location)
# For demo, we'll place the marker at the stop that would match the current time
now = datetime.now()
closest_stop = trip_stops.iloc[(trip_stops["sim_time"] - now).abs().argsort()[:1]]

for _, row in closest_stop.iterrows():
    folium.Marker(
        location=[row["stop_lat"], row["stop_lon"]],
        popup=f"Vehicle now at: {row['stop_name']}<br>Time: {now.strftime('%H:%M')}",
        icon=folium.Icon(color="red", icon="train")
    ).add_to(sim_map)

In [12]:
# Save map
map_path = "../maps/budapest_transportation_simulation.html"
sim_map.save(map_path)
map_path

'../maps/budapest_transportation_simulation.html'